In [1]:
%load_ext autoreload
%env PYTHONPATH=../

import sys
sys.path.append("../")
import pprint
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

import numpy as np
from tensorflow import keras
from oodeel.methods import MLS, DKNN, ODIN
from oodeel.methods import DKNN
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from oodeel.eval.metrics import bench_metrics, get_curve
from oodeel.datasets import OODDataset

from sklearn.metrics import *

import warnings
warnings.filterwarnings("ignore")

pp = pprint.PrettyPrinter()

env: PYTHONPATH=../


/Users/paul/workspace/oodeel/oodeel_dev_env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Two datasets experiment

In [2]:
%autoreload 2

oods_in = OODDataset('mnist', split="test")
oods_out = OODDataset('fashion_mnist', split="test")
oods_fit = OODDataset('mnist', split="train")

def preprocess_fn(*inputs):
    x = inputs[0] / 255
    return tuple([x] + list(inputs[1:]))


batch_size = 128
ds_in = oods_in.prepare(batch_size=batch_size, preprocess_fn=preprocess_fn)
ds_out = oods_out.prepare(batch_size=batch_size, preprocess_fn=preprocess_fn)
ds_fit = oods_fit.prepare(batch_size=batch_size, preprocess_fn=preprocess_fn, shuffle=True)

Metal device set to: AMD Radeon Pro 5500M

systemMemory: 64.00 GB
maxCacheSize: 3.99 GB

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [5]:

from oodeel.models.training_funs import train_convnet_classifier

try:
    model = tf.keras.models.load_model("../saved_models/mnist_model")
except OSError:
    

    train_config = {
        "input_shape": (28, 28, 1),
        "num_classes": 10,
        "is_prepared": True,
        "batch_size": 128,
        "epochs": 5,
        "save_dir": "../saved_models/mnist_model",
        "validation_data": ds_in #ds_in is actually the test set of MNIST
    }

    model = train_convnet_classifier(ds_fit, **train_config) #ds_fit is actually the train set of MNIST

Epoch 1/5
469/469 [==============================] - ETA: 0s - loss: 0.3287 - accuracy: 0.9023

INFO:tensorflow:Assets written to: ../saved_models/mnist_model/assets


INFO:tensorflow:Assets written to: ../saved_models/mnist_model/assets


469/469 [==============================] - 16s 32ms/step - loss: 0.3287 - accuracy: 0.9023 - val_loss: 0.0865 - val_accuracy: 0.9729
Epoch 2/5
469/469 [==============================] - ETA: 0s - loss: 0.0977 - accuracy: 0.9705

INFO:tensorflow:Assets written to: ../saved_models/mnist_model/assets


INFO:tensorflow:Assets written to: ../saved_models/mnist_model/assets


469/469 [==============================] - 13s 28ms/step - loss: 0.0977 - accuracy: 0.9705 - val_loss: 0.0555 - val_accuracy: 0.9828
Epoch 3/5
469/469 [==============================] - ETA: 0s - loss: 0.0783 - accuracy: 0.9766

INFO:tensorflow:Assets written to: ../saved_models/mnist_model/assets


INFO:tensorflow:Assets written to: ../saved_models/mnist_model/assets


469/469 [==============================] - 14s 30ms/step - loss: 0.0783 - accuracy: 0.9766 - val_loss: 0.0513 - val_accuracy: 0.9843
Epoch 4/5
469/469 [==============================] - 10s 22ms/step - loss: 0.0747 - accuracy: 0.9778 - val_loss: 0.0510 - val_accuracy: 0.9839
Epoch 5/5
469/469 [==============================] - 11s 24ms/step - loss: 0.0734 - accuracy: 0.9776 - val_loss: 0.0509 - val_accuracy: 0.9835


### MLS

In [6]:

%autoreload 2


oodmodel = MLS()
oodmodel.fit(model)
scores_in = oodmodel.score(ds_in)
scores_out = oodmodel.score(ds_out)


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

pp.pprint(metrics)


{'accuracy_score': 0.92835,
 'auroc': 0.9792125049999999,
 'fpr95tpr': 0.0613,
 'roc_auc_score': 0.97921253}


### DKNN

In [7]:
%autoreload 2


oodmodel = DKNN()
oodmodel.fit(model, ds_fit.take(10000))
scores_in = oodmodel.score(ds_in.take(1000))
scores_out = oodmodel.score(ds_out.take(1000))


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

pp.pprint(metrics)

{'accuracy_score': 0.5,
 'auroc': 0.9984382249999999,
 'fpr95tpr': 0.0061,
 'roc_auc_score': 0.998438245}


### ODIN

In [8]:

%autoreload 2
from oodeel.methods import ODIN

oodmodel = ODIN()
oodmodel.fit(model)
scores_in = oodmodel.score(ds_in)
scores_out = oodmodel.score(ds_out)


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

pp.pprint(metrics)

{'accuracy_score': 0.5,
 'auroc': 0.9909027650000001,
 'fpr95tpr': 0.0372,
 'roc_auc_score': 0.990903005}


## Single dataset experiment

(Leave-$k$-classes-out training).
First, we need to define a training function

In [9]:

%autoreload 2

oods_test = OODDataset('mnist', split="test")
oods_train = OODDataset('mnist', split="train")

batch_size = 128
inc_labels = [0, 1, 2, 3, 4]
oods_train, _ = oods_train.assign_ood_labels_by_class(in_labels=inc_labels)
oods_in, oods_out = oods_test.assign_ood_labels_by_class(in_labels=inc_labels)


def preprocess_fn(*inputs):
    x = inputs[0] / 255
    return tuple([x] + list(inputs[1:]))

ds_train = oods_train.prepare(batch_size=batch_size, preprocess_fn=preprocess_fn, shuffle=True)
ds_in = oods_in.prepare(batch_size=batch_size, with_ood_labels=False, preprocess_fn=preprocess_fn)
ds_out = oods_out.prepare(batch_size=batch_size, with_ood_labels=False, preprocess_fn=preprocess_fn)


In [10]:
%autoreload 2
from oodeel.models.training_funs import train_convnet_classifier

train_config = {
    "input_shape": (28, 28, 1),
    "num_classes": len(inc_labels),
    "is_prepared": True,
    "batch_size": 128,
    "epochs": 5
}

model = train_convnet_classifier(ds_train, **train_config)

Epoch 1/5
240/240 [==============================] - 9s 32ms/step - loss: 0.2136 - accuracy: 0.9347
Epoch 2/5
240/240 [==============================] - 6s 24ms/step - loss: 0.0500 - accuracy: 0.9850
Epoch 3/5
240/240 [==============================] - 7s 29ms/step - loss: 0.0411 - accuracy: 0.9874
Epoch 4/5
240/240 [==============================] - 5s 20ms/step - loss: 0.0391 - accuracy: 0.9874
Epoch 5/5
240/240 [==============================] - 6s 23ms/step - loss: 0.0386 - accuracy: 0.9883


## MLS

In [11]:
%autoreload 2

oodmodel = MLS()
oodmodel.fit(model)
scores_in = oodmodel.score(ds_in)
scores_out = oodmodel.score(ds_out)


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

pp.pprint(metrics)

{'accuracy_score': 0.8827,
 'auroc': 0.9456968323399055,
 'fpr95tpr': 0.25179996108192254,
 'roc_auc_score': 0.9456956314117804}


### DKNN

In [12]:
%autoreload 2

oodmodel = DKNN()
oodmodel.fit(model, ds_train.take(10000))
scores_in = oodmodel.score(ds_in.take(1000))
scores_out = oodmodel.score(ds_out.take(1000))


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

pp.pprint(metrics)

{'accuracy_score': 0.4861,
 'auroc': 0.967651319645875,
 'fpr95tpr': 0.14127262113251604,
 'roc_auc_score': 0.9676495582846246}


### ODIN

In [13]:
%autoreload 2

#x_test, y_id = data_handler.convert_to_numpy(x_id)

oodmodel = ODIN()
oodmodel.fit(model)
scores_in = oodmodel.score(ds_in)
scores_out = oodmodel.score(ds_out)


metrics = bench_metrics(
    (scores_in, scores_out),  
    metrics = ["auroc", "fpr95tpr", accuracy_score, roc_auc_score], 
    threshold = -5
    )

pp.pprint(metrics)

{'accuracy_score': 0.4861,
 'auroc': 0.9244589988927043,
 'fpr95tpr': 0.3957968476357268,
 'roc_auc_score': 0.9244590589391105}
